In [1]:
import java.util.{SplittableRandom => Random}
import scala.collection.immutable.TreeMap
import scala.annotation.tailrec

import java.util.{SplittableRandom => Random}
import scala.collection.immutable.TreeMap
import scala.annotation.tailrec

In [2]:
trait Generator[T] {
    
    def generate(size: Int, random: Random): T
    
    final def filter(predicate: T => Boolean): Generator[T] =
        new FilteredGenerator(this, predicate)
    
    final def sample(maxSize: Int = 10, count: Int = 1): Map[Int, List[T]] = {
        val random = new Random()
        val samples =
            for (size <- 0 to maxSize)
            yield size -> List.fill(count)(generate(size, random))
        TreeMap(samples: _*)
    }
    
    
}

final class FilteredGenerator[T](
    underlying: Generator[T], predicate: T => Boolean) extends Generator[T] {
    
    @tailrec def generate(size: Int, random: Random): T = {
        val candidate = underlying.generate(size, random)
        if (predicate(candidate))
            candidate
        else
            generate(size, random)
    }
    
}

object Generator {
    
    implicit def int: Generator[Int] = new Generator[Int] {
        def generate(size: Int, random: Random): Int = {
            val min = -size
            val max = size + 1
            random.nextInt(min, max) // yolo
        }
    }
    
    
}

defined trait Generator
defined class FilteredGenerator
defined object Generator

In [3]:
Generator.int.filter(_ >= 0).sample(count = 10)

res2: Map[Int, List[Int]] = Map(
  0 -> List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
  1 -> List(1, 1, 1, 1, 1, 1, 0, 1, 0, 0),
  2 -> List(1, 0, 2, 0, 1, 1, 1, 1, 0, 1),
  3 -> List(2, 2, 0, 3, 1, 3, 0, 1, 0, 2),
  4 -> List(0, 1, 0, 2, 1, 1, 2, 4, 3, 4),
  5 -> List(5, 2, 3, 1, 1, 4, 2, 0, 2, 2),
  6 -> List(2, 3, 3, 0, 3, 5, 1, 6, 1, 4),
  7 -> List(1, 1, 7, 3, 1, 1, 7, 2, 5, 3),
  8 -> List(3, 5, 5, 1, 8, 7, 1, 7, 2, 5),
  9 -> List(3, 7, 7, 6, 2, 1, 2, 1, 3, 3),
  10 -> List(5, 3, 10, 3, 3, 10, 10, 9, 5, 5)
)

In [4]:
object Result {
    def apply(b: Boolean): Result = b match {
        case true => Success
        case false => Failure(Nil)
    }

    def merge(rs: List[Result]): Result =
        rs.foldLeft(Success: Result) {
            case (Failure(cs), _) => Failure(cs)
            case (Success, Success) => Success
            case (Success, Failure(cs)) => Failure(cs)
        }
}

sealed trait Result
object Success extends Result
case class Failure(counterexamples: List[Any] /* yolo */) extends Result

object Property {
    
    implicit def boolean(b: Boolean): Property = new Property {
        def run(size: Int, random: Random): Result =
            if (b)
                Success
            else
                Failure(Nil)
    }
    
    def forAll[T](prop: T => Property)(implicit generator: Generator[T]): Property =
        new Property {
            def run(size: Int, random: Random): Result = {
                val input = generator.generate(size, random)
                val subprop: Property = prop(input)
                subprop.run(size, random) match {
                    case Success => Success
                    case Failure(counterexamples) => Failure(input :: counterexamples)
                }
            }
        }
    
    
}


trait Property {
    def run(size: Int, random: Random): Result
    
    final def check: Unit = {
        val random = new Random()
        val rs = for (size <- 0 to 100) yield run(size, random)
        Result.merge(rs.toList) match {
            case Success => Console.out.println("Property successfully checked")
            case Failure(cs) => Console.err.println(s"Property failed with counterexample: ${cs.mkString("(", ", ", ")")}")
        }
    }
}

defined object Result
defined trait Result
defined object Success
defined class Failure
defined object Property
defined trait Property

In [5]:
(false: Property).check

Property failed with counterexample: ()


In [6]:
Property.forAll { (x: Int) =>
    x != x
}.check

Property failed with counterexample: (0)


In [7]:
Property.forAll { (x: Int) =>
    Property.forAll { (y: Int) =>
        x + y == y + x
    }
}.check

Property successfully checked
